In [2]:
! pip install jupyter-dash


   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------- ----------------------------- 2.1/7.8 MB 11.8 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/7.8 MB 11.7 MB/s eta 0:00:01
   ---------------------------------- ----- 6.8/7.8 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ------ --------------------------------- 2.4/14.8 MB 12.2 MB/s eta 0:00:02
   ------------- -------------------------- 5.0/14.8 MB 12.1 MB/s eta 0:00:01
   ------------------- -------------------- 7.3/14.8 MB 11.9 MB/s eta 0:00:01
   ------------------------ --------------- 9.2/14.8 MB 11.4 MB/s eta 0:00:01
   ------------------------------- -------- 11.8/14.8 MB 11.0 MB/s eta 0:00:01
   ------------------------------------- -- 13.9/14.8 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 14.8/14.8 MB 10.5 MB/s eta 0:00:00
  At


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
! pip install dash-uploader


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dash-uploader: filename=dash_uploader-0.6.1-py3-none-any.whl size=70337 sha256=738fcab1576d6a36f4882ddc596c81c2b71dcaf42c24770ea2acf1b695a8f8d6
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\cd\95\15\7d22467b2c80900e3224e510b40b0c0bfa0c6a32af6a5eb673
Successfully built dash-uploader
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
! pip install opencv-python


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   -- ------------------------------------- 2.1/39.5 MB 10.7 MB/s eta 0:00:04
   ---- ----------------------------------- 4.2/39.5 MB 10.5 MB/s eta 0:00:04
   ------ --------------------------------- 6.3/39.5 MB 10.4 MB/s eta 0:00:04
   -------- ------------------------------- 8.7/39.5 MB 10.3 MB/s eta 0:00:03
   ---------- ----------------------------- 10.7/39.5 MB 10.3 MB/s eta 0:00:03
   ------------- -------------------------- 12.8/39.5 MB 10.2 MB/s eta 0:00:03
   --------------- ------------------------ 14.9/39.5 MB 10.2 MB/s eta 0:00:03
   ----------------- ---------------------- 17.3/39.5 MB 10.2 MB/s eta 0:00:03
   ------------------- -------------------- 19.4/39.5 MB 10.2 MB/s eta 0:00:02
   --------------------- ------------------ 21.5/39.5 MB 10.1 MB/s eta 0:00:02
   ------------------------ --------------- 23.9/39.5 MB 10.1 MB/s eta 0:00:02
   -------------------------- ------------- 26.0/39.5 MB 10.2 MB/


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
! pip install tensorflow



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import dash
from dash import dcc, html, Input, Output, State
import dash_uploader as du
import os
import tensorflow as tf
import numpy as np
import cv2
import base64
from io import BytesIO
from PIL import Image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

# Load pre-trained VGG16 model
model = VGG16(weights='imagenet')

# Initialize Dash app
app = dash.Dash(__name__)
du.configure_upload(app, "./uploads")

app.layout = html.Div([
    html.H1("VGG16 Image Classification App"),
    dcc.Upload(
        id='upload-image',
        children=html.Button('Upload Image'),
        multiple=False
    ),
    html.Button("Predict", id="predict-button", n_clicks=0),
    html.Img(id="uploaded-image", style={"maxWidth": "300px", "display": "none"}),
    html.Div(id="prediction-output")
])

@app.callback(
    [Output("uploaded-image", "src"), Output("uploaded-image", "style")],
    [Input("upload-image", "contents")]
)
def display_image(contents):
    if contents:
        return contents, {"maxWidth": "300px", "display": "block"}
    return None, {"display": "none"}

@app.callback(
    Output("prediction-output", "children"),
    [Input("predict-button", "n_clicks")],
    [State("upload-image", "contents")]
)
def predict(n_clicks, contents):
    if n_clicks > 0 and contents:
        try:
            # Convert base64 image to PIL Image
            encoded_image = contents.split(",")[1]
            decoded_image = base64.b64decode(encoded_image)
            image = Image.open(BytesIO(decoded_image))
            image = image.resize((224, 224))
            image = np.array(image)
            if image.shape[-1] == 4:
                image = image[:, :, :3]  # Convert RGBA to RGB
            image = np.expand_dims(image, axis=0)
            image = preprocess_input(image)
            
            # Predict
            preds = model.predict(image)
            decoded_preds = decode_predictions(preds, top=3)[0]
            results = [f"{i+1}. {label} ({prob:.2f})" for i, (_, label, prob) in enumerate(decoded_preds)]
            return html.Ul([html.Li(result) for result in results])
        except Exception as e:
            return f"Error processing image: {str(e)}"
    return "Upload an image and click Predict."

if __name__ == "__main__":
    app.run_server(debug=True)
